In [15]:
%%capture
!pip install gdown

In [16]:
import gdown

URL = 'https://drive.google.com/drive/folders/1CZ48RwnagQgZkvrWhJMoWNnLAwlWRUec?usp=sharing'
gdown.download_folder(URL, quiet=True)

['/content/buildings/building_heights.csv',
 '/content/buildings/buildings.csv',
 '/content/buildings/custom_buildings.csv']

In [17]:
import pandas as pd
from ast import literal_eval

building_heights_df = pd.read_csv('buildings/building_heights.csv')

custom_buildings_df = pd.read_csv('buildings/custom_buildings.csv')
osm_buildings_df = pd.read_csv('buildings/buildings.csv')

buildings_df = pd.concat([custom_buildings_df, osm_buildings_df], ignore_index=True)
df = pd.merge(buildings_df, building_heights_df, on='way_id', how='inner')

df['building_border'] = df['building_border'].apply(literal_eval)
df.head()

,way_id,building_border,height
0,11,"[(20.9985463, 105.8676339), (20.9991551, 105.8...",6.750650
1,1,"[(20.9952857, 105.8636886), (20.995253, 105.86...",6.616604
2,2,"[(20.9968625, 105.8644855), (20.997205, 105.86...",7.163128
3,3,"[(20.9930575, 105.8636499), (20.9930194, 105.8...",6.543596
4,4,"[(20.9952454, 105.8702933), (20.9929813, 105.8...",3.415808


In [19]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt

fig = go.Figure()

# Generate a colormap for unique colors
num_traces = len(df)
colors = plt.cm.get_cmap('tab20', num_traces)  # Use 'tab20' for up to 20 distinct colors
colors = ['rgb({},{},{})'.format(int(c[0] * 255), int(c[1] * 255), int(c[2] * 255)) for c in colors.colors]

for i, row in df.iterrows():
    latitudes, longitudes = zip(*row['building_border'])
    height = row['height']

    x = list(longitudes)
    y = list(latitudes)
    z = [height] * len(x)

    x_base = list(longitudes)
    y_base = list(latitudes)
    z_base = [0] * len(x)

    fig.add_trace(go.Mesh3d(
        x=x + x_base,
        y=y + y_base,
        z=z + z_base,
        color=colors[i],
        opacity=0.5,
        flatshading=True,
        alphahull=0
    ))

fig.update_layout(
    scene=dict(
        xaxis_title='Longitude',
        yaxis_title='Latitude',
        zaxis_title='Height',
        aspectmode="cube"
    ),
)

fig.show()

<ipython-input-19-b7c720cc0427>:8: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.

